### 도구 호출

도구 호출은 모델에게 외부 기능과 API에 접근할 수 있는 능력을 제공하는 기능입니다. 이 노트북에서는 Nova 모델과 함께 이 기능을 사용하는 개념과 모범 사례를 살펴보겠습니다.

### 스키마 정의하기

"도구"라고 할 때, 우리는 실제 코드를 실행하는 함수를 의미합니다. 이 함수에 대한 세부 정보를 제공하기 위해, 모델에 도구 구성을 제공합니다. 이 도구 구성에는 이름, 설명 및 매개변수에 대한 세부 정보가 포함됩니다.

계산기 도구는 다음과 같이 정의될 수 있습니다:
```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "calculator", # 도구의 이름
                "description": "수학 방정식을 실행할 수 있는 계산기 도구",
                "inputSchema": {
                    "json": { 
                        "type": "object", # 최상위 스키마는 반드시 "object" 타입이어야 하며, properties와 required 키가 있어야 합니다. 이 레벨에서는 다른 필드가 허용되지 않습니다
                        "properties": {
                            "equation": { # 매개변수의 이름
                                "type": "string", # 매개변수 타입: 문자열/정수/기타
                                "description": "평가할 전체 방정식" # 매개변수에 대한 유용한 설명
                            }
                        },
                        "required": [ # 필수 매개변수 목록
                            "equation"
                        ]
                    }
                }
            }
        }
    ]
}
```

또는 SEC 파일을 검색할 수 있는 검색기 도구:

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "secRetriever", 
                "description": "데이터베이스에서 SEC 파일에 접근할 수 있는 검색기",
                "inputSchema": {
                    "json": { 
                        "type": "object", # 최상위 스키마는 반드시 "object" 타입이어야 하며, properties와 required 키가 있어야 합니다. 이 레벨에서는 다른 필드가 허용되지 않습니다
                        "properties": {
                            "query": { 
                                "type": "string", 
                                "description": "검색할 전체 쿼리" 
                            },
                            "ticker": { 
                                "type": "string", 
                                "description": "회사의 주식 티커"
                            },
                            "year": { 
                                "type": "string", 
                                "description": "파일의 관련 연도"
                            }
                        },
                        "required": [ # year가 제공되지 않은 것은 선택적 매개변수임을 나타냅니다
                            "query",
                            "ticker"
                        ]
                    }
                }
            }
        }
    ]
}
```

또는 두 정수를 받아 곱하는 곱셈 도구:

```python
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "두 숫자를 서로 곱합니다",
                "inputSchema": {
                    "json": { 
                        "type": "object", # 최상위 스키마는 반드시 "object" 타입이어야 하며, properties와 required 키가 있어야 합니다. 이 레벨에서는 다른 필드가 허용되지 않습니다
                        "properties": {
                            "int1": { 
                                "type": "integer", 
                                "description": "곱할 첫 번째 숫자" 
                            },
                            "int2": { 
                                "type": "integer", 
                                "description": "곱할 두 번째 숫자"
                            }
                        },
                        "required": [
                            "int1",
                            "int2"
                        ]
                    }
                }
            }
        }
    ]
}

```

### Nova에 도구 제공하기
이 예제들에서는 converse API를 사용하고 모델의 toolConfig 매개변수를 통해 도구를 모델에 전달할 수 있습니다. 도구 호출을 활용할 때는 "탐욕적 디코딩" 매개변수를 활용하는 것이 좋습니다. Nova에서는 temperature, topP 및 topK를 1로 설정하여 이를 수행합니다.

먼저 계산기부터 시작합니다:

In [1]:
import boto3

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

system = [
    {
        "text": "수학 방정식을 풀 때는 항상 계산기 도구를 사용해야 하며, 매개변수적 지식을 사용해서는 안 됩니다."
    }
]


messages = [{"role": "user", "content": [{"text": "2+2는 얼마인가요?"}]}]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "calculator",
                "description": "x",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "equation": {
                                "type": "string",
                                "description": "The full equation to evaluate",
                            }
                        },
                        "required": ["equation"],
                    }
                },
            }
        }
    ]
}

inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

initial_response = client.converse(
    modelId=PRO_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)

tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

{'toolUseId': 'tooluse_ITzHzxtxTd6iQYwMWo2tyA', 'name': 'calculator', 'input': {'equation': '2+2'}}


검색기:

In [2]:
import boto3

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

system = [
    {
        "text": "재정 관련 질문에 대해서는 항상 검색 도구(retriever tool)를 사용해야 하며, 매개변수적 지식을 사용해서는 안 됩니다."
    }
]


messages = [
    {
        "role": "user",
        "content": [{"text": "What was Amazon's reported revenue in 2023"}],
    }
]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "secRetriever",
                "description": "A retriever that can access SEC filings from a database",
                "inputSchema": {
                    "json": {
                        "type": "object",  # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The full query to search for",
                            },
                            "ticker": {
                                "type": "string",
                                "description": "The stock ticker of the company",
                            },
                            "year": {
                                "type": "string",
                                "description": "The relevant year of the filings",
                            },
                        },
                        "required": [  # Note that year is not provided, this indicates it's an optional parameter
                            "query",
                            "ticker",
                        ],
                    }
                },
            }
        }
    ]
}

inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}


initial_response = client.converse(
    modelId=PRO_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)
tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

{'toolUseId': 'tooluse_CfnAp4shTcyJs7vr5F_DtA', 'name': 'secRetriever', 'input': {'ticker': 'AMZN', 'year': '2023', 'query': 'revenue'}}


그리고 마지막으로 곱셈 도구

In [4]:
import boto3

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")

messages = [{"role": "user", "content": [{"text": "What is 2*5"}]}]

system = [
    {
        "text": "For multiplication questions, you must always use the multiply tool and not your parametric knowledge"
    }
]

tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "multiply",
                "description": "Multiplies two numbers together",
                "inputSchema": {
                    "json": {
                        "type": "object",  # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "int1": {
                                "type": "int",
                                "description": "The first number to multiply",
                            },
                            "int2": {
                                "type": "int",
                                "description": "The second number to multiply",
                            },
                        },
                        "required": ["int1", "int2"],
                    }
                },
            }
        }
    ]
}
inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

initial_response = client.converse(
    modelId=PRO_MODEL_ID,
    system=system,
    messages=messages,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
    toolConfig=tool_config,
)
tool_use = next(
    block["toolUse"]
    for block in initial_response["output"]["message"]["content"]
    if "toolUse" in block
)

print(tool_use)

{'toolUseId': 'tooluse_Fhz0dTO6QaSQyAkmP9uXtw', 'name': 'multiply', 'input': {'int2': 5, 'int1': 2}}


### 도구 선택 사용하기

Amazon Nova 모델에서는 "도구 선택" API 매개변수를 활용할 수 있습니다. 도구 선택 매개변수를 사용하면 모델이 도구를 선택할 때의 동작을 제어할 수 있습니다. 세 가지 옵션이 있습니다:

**Tool**: 지정된 도구가 한 번 호출됩니다

```python
"toolChoice": {
   "tool": { "name" : <tool_name> }
}
```

**Any**: 제공된 도구 중 하나가 최소 한 번 이상 호출됩니다

```python
"toolChoice": {
   "any": {}
}
```

**Auto**: 모델이 도구를 호출할지 여부를 결정합니다. 필요한 경우 여러 도구가 호출될 수 있습니다

```python
"toolChoice": {
   "auto": {}
}
```

#### 도구 선택 - 도구

"도구" 옵션은 구조화된 출력과 같이 모델이 매번 동일한 도구를 호출하도록 강제하고 싶은 사용 사례에서 일반적입니다.

In [5]:
import boto3

with open("media/nutritional_benifits.png", "rb") as media_file:
    binary_data = media_file.read()

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

tool_config = {
    "toolChoice": {"tool": {"name": "print_nutrition_info"}},
    "tools": [
        {
            "toolSpec": {
                "name": "print_nutrition_info",
                "description": "Extracts nutrition information from an image of a nutrition label",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "calories": {
                                "type": "integer",
                                "description": "The number of calories per serving",
                            },
                            "total_fat": {
                                "type": "integer",
                                "description": "The amount of total fat in grams per serving",
                            },
                            "cholesterol": {
                                "type": "integer",
                                "description": "The amount of cholesterol in milligrams per serving",
                            },
                            "total_carbs": {
                                "type": "integer",
                                "description": "The amount of total carbohydrates in grams per serving",
                            },
                            "protein": {
                                "type": "integer",
                                "description": "The amount of protein in grams per serving",
                            },
                        },
                        "required": [
                            "calories",
                            "total_fat",
                            "cholesterol",
                            "total_carbs",
                            "protein",
                        ],
                    }
                },
            }
        },
    ],
}

messages = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "png",
                    "source": {"bytes": binary_data},
                }
            },
            {
                "text": "Please print the nutrition information from this nutrition label image"
            },
        ],
    }
]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")

response = client.converse(
    modelId=PRO_MODEL_ID,
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(
    next(
        block["toolUse"]
        for block in response["output"]["message"]["content"]
        if "toolUse" in block
    )
)

{'toolUseId': 'tooluse_nzSldwKYTA-B7E9w8nx68A', 'name': 'print_nutrition_info', 'input': {'total_carbs': 35, 'total_fat': 5, 'protein': 9, 'cholesterol': 15, 'calories': 220}}


#### 도구 선택 - Any

일부 사용 사례에서는 사용자 쿼리의 맥락에 관계없이 항상 도구가 호출되어야 합니다.

In [9]:
import boto3

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

tool_config = {
    "toolChoice": {"any": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "get_the_weather",
                "description": "API to get the current weather",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "The city to get the weather for. If unknown prompt the user for more information",
                            },
                        },
                        "required": ["city"],
                    }
                },
            }
        },
        {
            "toolSpec": {
                "name": "follow_up_question",
                "description": "Ask a follow up question to the user",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {
                                "type": "string",
                                "description": "Question to ask the user",
                            },
                        },
                        "required": ["question"],
                    }
                },
            }
        },
    ],
}

system = [
    {
        "text": "You can get the current weather for the user, if you need more information use the follow_up_question tool"
    }
]


messages = [
    {
        "role": "user",
        "content": [
            {"text": "Can you get the weather in California"},
        ],
    }
]

# messages = [
#     {
#         "role": "user",
#         "content": [
#             {"text": "3+5는 얼마인가요?"},
#         ],
#     }
# ]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")


response = client.converse(
    modelId=PRO_MODEL_ID,
    system=system,
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(
    next(
        block["toolUse"]
        for block in response["output"]["message"]["content"]
        if "toolUse" in block
    )
)

{'toolUseId': 'tooluse_MjIBwYTgQ5KMxarSLBnPEA', 'name': 'follow_up_question', 'input': {'question': 'California is a large state with many cities. Could you please specify which city you would like the weather for?'}}


#### 도구 선택 - Auto

도구가 항상 필요하지 않은 사용 사례의 경우 도구 선택을 자동으로 설정할 수 있습니다. 이것이 기본 동작이며 도구 선택을 완전히 모델에 맡깁니다.

In [7]:
import boto3

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

tool_config = {
    "toolChoice": {"auto": {}},
    "tools": [
        {
            "toolSpec": {
                "name": "search",
                "description": "API that provides access to the internet",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Query to search by",
                            },
                        },
                        "required": ["query"],
                    }
                },
            }
        }
    ],
}

messages = [
    {
        "role": "user",
        "content": [
            {"text": "Who was in the cast of Wicked"},
        ],
    }
]

system = [{"text": "You are a helpful chatbot. You can use a tool if necessary"}]

inf_params = {"topP": 1, "temperature": 1}

client = boto3.client("bedrock-runtime", region_name="us-east-1")

response = client.converse(
    modelId=PRO_MODEL_ID,
    messages=messages,
    toolConfig=tool_config,
    inferenceConfig=inf_params,
    additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
)

print(response["output"]["message"]["content"])

[{'text': '<thinking> To answer the question about the cast of Wicked, I need to search for information about the original Broadway cast of the musical. </thinking>\n'}, {'toolUse': {'toolUseId': 'tooluse_8i9Ov6bORvOVZO98mx4HHA', 'name': 'search', 'input': {'query': 'original Broadway cast of Wicked'}}}]
